In [ ]:
from arcgis.features import FeatureLayer
import json
import tempfile
import arcpy
import os

# Step 1: Define the service URL and folder directory
service_url = 'map service url with layer id'
folder_dir = r'C:'

# Step 2: Create a feature layer object using the service URL
feature_layer = FeatureLayer(service_url)

# Step 3: Query the feature layer to get the features with geometry
print(f"Querying {feature_layer.url}")
query_result = feature_layer.query(return_geometry=True)

# Step 4: Extract the features from the query result
print('Grabbing Features')
features = query_result.features
print(f"Extracting {len(features)} features")

# Step 5: Convert the query result to JSON format
json_str = query_result.to_json
data = json.loads(json_str)

# Step 6: Create a temporary directory to store the JSON file
temp_dir = tempfile.mkdtemp()

# Step 7: Write the JSON data to a temporary file
json_file = os.path.join(temp_dir, 'data.json')
with open(json_file, "w") as outfile:
    json.dump(data, outfile, indent=2)
print(f"JSON file created at {json_file}")

# Step 8: Create a File Geodatabase (FGDB) and Feature Class (FC)
# Replace spaces in feature layer name with underscores for FC name
fc_name = feature_layer.properties.name.replace(' ', '_')
print(f"Creating MapServiceData file geodatabase")

# Get the geometry type from the feature layer properties
geom = feature_layer.properties.geometryType.replace('esriGeometry', '').upper()

# Define the name and path for the FGDB
fgdb_name = "MapServiceData.gdb"
file_gdb = os.path.join(folder_dir, fgdb_name)

# Create the FGDB in the specified folder directory
arcpy.management.CreateFileGDB(folder_dir, fgdb_name)

# Set the workspace environment to the new FGDB
arcpy.env.workspace = file_gdb

# Step 9: Convert the JSON file to a feature class within the FGDB
print(f"Creating {fc_name} feature class")
arcpy.conversion.JSONToFeatures(json_file, fc_name, geom)
print(f"Data saved to {file_gdb}")